In [22]:
# The code was removed by DSX for sharing.

In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

Get the Wiki page and parse it with BeautifulSoup

In [116]:
wiki_toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content, 'lxml')

Extract postal codes, boroughs and neighborhoods and put them in a DataFrame

In [127]:
items = []
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    parts = [it for it in item.stripped_strings if it not in "()"]
    borough = parts[1]
    neighborhood = ' '.join(parts[2:])
    
    items.append((postal_code, borough, neighborhood))

df_toronto = pd.DataFrame([it for it in items], columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North)
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre 969 East...
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [128]:
df_toronto.shape

(103, 3)

In [129]:
# The code was removed by DSX for sharing.

In [130]:
locations = []

for item in df_toronto['PostalCode']:
        
    # construct URL to make API call
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address="Toronto, ON {}"'.format(API_key, item)

    response = requests.get(url).json() # get response
    
    if response['status'] == 'ZERO_RESULTS':
        print(item)
        continue
    
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    locations.append((item, latitude, longitude))
    
df_locations = pd.DataFrame([it for it in locations], columns=['PostalCode', 'Latitude', 'Longitude'])
df_locations.tail()

,PostalCode,Latitude,Longitude
98,M8X,43.653654,-79.506944
99,M4Y,43.665860,-79.383160
100,M7Y,43.662744,-79.321558
101,M8Y,43.636258,-79.498509
102,M8Z,43.628841,-79.520999


In [133]:
df_data = df_toronto.merge(df_locations, on="PostalCode", how='left')

df_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


In [134]:
!pip install geopy folium

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/e

In [135]:
# The code was removed by DSX for sharing.

In [136]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    total = len(names)
    count = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print('Unable to fetch: {0}'.format(name))
            continue

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        count += 1
        print("{0}/{1}".format(count, total))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [137]:
# type your answer here
toronto_venues = getNearbyVenues(df_data['Neighborhood'], df_data['Latitude'], df_data['Longitude'])

Parkwoods
1/103
Victoria Village
2/103
Regent Park / Harbourfront
3/103
Lawrence Manor / Lawrence Heights
4/103
Ontario Provincial Government
5/103
Islington Avenue
6/103
Malvern / Rouge
7/103
Don Mills North
8/103
Parkview Hill / Woodbine Gardens
9/103
Garden District , Ryerson
10/103
(Glencairn)
11/103
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)
12/103
Rouge Hill / Port Union / Highland Creek
13/103
Don Mills South Flemingdon Park
14/103
Woodbine Heights
15/103
St. James Town
16/103
Humewood-Cedarvale
17/103
(Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
18/103
Guildwood / Morningside / West Hill
19/103
The Beaches
20/103
(Berczy Park)
21/103
Caledonia-Fairbanks
22/103
Woburn
23/103
Leaside
24/103
(Central Bay Street
25/103
(Christie)
26/103
Cedarbrae
27/103
Hillcrest Village
28/103
Bathurst Manor / Wilson Heights / Downsview North)
29/103
Thorncliffe Park
30/103
Richmond / Adelaide / King
31/103
Dufferin / Dovercourt Village
32/103


In [138]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [139]:
project.save_data('df_data.p', pickle.dumps(df_data), overwrite=True)
project.save_data('toronto_venues.p', pickle.dumps(toronto_venues), overwrite=True)
project.save_data('df_locations.p', pickle.dumps(df_locations), overwrite=True)
project.save_data('df_toronto.p', pickle.dumps(df_toronto), overwrite=True)

{'asset_id': '4420dbc1-4e6d-4b6e-958f-22753043ee6b',
 'bucket_name': 'courseracapstone-donotdelete-pr-duvaojegpzermh',
 'file_name': 'df_toronto.p',
 'message': 'File df_toronto.p has been written successfully to the associated OS'}

In [140]:
toronto_venues_file = project.get_file('toronto_venues.p')
toronto_venues = pickle.load(toronto_venues_file)

df_data = pickle.load(project.get_file('df_data.p'))

In [141]:
print(toronto_venues.shape)

(1304, 7)


In [142]:
toronto_venues[toronto_venues['Neighborhood'].str.contains('Toronto')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
458,East Toronto The Danforth East),43.685347,-79.338106,The Path,43.683923,-79.335007,Park
459,East Toronto The Danforth East),43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store
460,East Toronto The Danforth East),43.685347,-79.338106,Donlands Subway Station,43.680960,-79.337759,Metro Station
461,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
462,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,Roundhouse Park,43.641745,-79.384279,Park
463,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,Lake Ontario,43.639398,-79.379589,Lake
464,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,BeaverTails Toronto Waterfront,43.639899,-79.380197,Fast Food Restaurant
465,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,iQ Food Co,43.642851,-79.382081,Salad Place
466,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,Longo's Maple Leaf Square,43.642517,-79.381393,Supermarket
467,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,Real Sports Apparel,43.642860,-79.380184,Sporting Goods Shop


In [144]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [145]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,(Berczy Park),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,(Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0
2,(Christie),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,(Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,(Del Ray / Mount Dennis / Keelsdale and Silver...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----(Berczy Park)----
                venue  freq
0              Bakery  0.07
1            Beer Bar  0.07
2  Seafood Restaurant  0.07
3      Farmers Market  0.07
4        Cocktail Bar  0.07


----(Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.07
2                 Spa  0.07
3  Chinese Restaurant  0.07
4     Thai Restaurant  0.03


----(Christie)----
                venue  freq
0                Café  0.20
1       Grocery Store  0.20
2                Park  0.13
3  Italian Restaurant  0.07
4               Diner  0.07


----(Clarks Corners / Tam O'Shanter / Sullivan----
                venue  freq
0         Pizza Place  0.22
1         Coffee Shop  0.11
2  Chinese Restaurant  0.11
3     Thai Restaurant  0.11
4        Noodle House  0.11


----(Del Ray / Mount Dennis / Keelsdale and Silverthorn----
                   venue  freq
0            Coffee Shop   0.2
1     Turkish Restaurant   0.2
2         Discount Store   0.2
3  Check Cashing

In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,(Berczy Park),Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Belgian Restaurant,Bistro,Steakhouse,Bagel Shop,Basketball Stadium
1,(Central Bay Street,Coffee Shop,Spa,Chinese Restaurant,Italian Restaurant,Gastropub,Art Museum,Seafood Restaurant,Japanese Restaurant,Bubble Tea Shop,Ramen Restaurant
2,(Christie),Grocery Store,Café,Park,Coffee Shop,Convenience Store,Diner,Italian Restaurant,Baby Store,Restaurant,Nightclub
3,(Clarks Corners / Tam O'Shanter / Sullivan,Pizza Place,Pharmacy,Thai Restaurant,Noodle House,Chinese Restaurant,Italian Restaurant,Fried Chicken Joint,Coffee Shop,Garden Center,Garden
4,(Del Ray / Mount Dennis / Keelsdale and Silver...,Coffee Shop,Check Cashing Service,Turkish Restaurant,Discount Store,Sandwich Place,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run


In [156]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [157]:
toronto_merged = toronto_grouped

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,(Berczy Park),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Belgian Restaurant,Bistro,Steakhouse,Bagel Shop,Basketball Stadium
1,(Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Spa,Chinese Restaurant,Italian Restaurant,Gastropub,Art Museum,Seafood Restaurant,Japanese Restaurant,Bubble Tea Shop,Ramen Restaurant
2,(Christie),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Grocery Store,Café,Park,Coffee Shop,Convenience Store,Diner,Italian Restaurant,Baby Store,Restaurant,Nightclub
3,(Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Pizza Place,Pharmacy,Thai Restaurant,Noodle House,Chinese Restaurant,Italian Restaurant,Fried Chicken Joint,Coffee Shop,Garden Center,Garden
4,(Del Ray / Mount Dennis / Keelsdale and Silver...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Check Cashing Service,Turkish Restaurant,Discount Store,Sandwich Place,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run


In [158]:
localization_data = df_data[['Neighborhood', 'Latitude', 'Longitude']]
localization_data.set_index('Neighborhood', inplace=True)
toronto_merged = toronto_merged.join(localization_data, on='Neighborhood')
toronto_merged.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,(Berczy Park),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Farmers Market,Beer Bar,Seafood Restaurant,Belgian Restaurant,Bistro,Steakhouse,Bagel Shop,Basketball Stadium,43.644771,-79.373306
1,(Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Chinese Restaurant,Italian Restaurant,Gastropub,Art Museum,Seafood Restaurant,Japanese Restaurant,Bubble Tea Shop,Ramen Restaurant,43.657952,-79.387383
2,(Christie),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Park,Coffee Shop,Convenience Store,Diner,Italian Restaurant,Baby Store,Restaurant,Nightclub,43.669542,-79.422564
3,(Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Thai Restaurant,Noodle House,Chinese Restaurant,Italian Restaurant,Fried Chicken Joint,Coffee Shop,Garden Center,Garden,43.781638,-79.304302
4,(Del Ray / Mount Dennis / Keelsdale and Silver...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Turkish Restaurant,Discount Store,Sandwich Place,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,43.691116,-79.476013


In [161]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinates of Toronto are 43.653963, -79.387207.
